In [1]:
import pandas as pd
import myFunction as mf
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
from sklearn.externals import joblib
#import joblib
from xgboost.sklearn import XGBClassifier
import random
from sklearn.utils import shuffle
from scipy import stats
import myFunction as mf
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('max_colwidth',None)

In [2]:
#train test data split
train_variables=['gender', 'age', 'vasopressin',  'urineoutput', 'heartrate_mean', 
                 'sysbp_mean', 'diasbp_mean', 'resprate_mean', 'tempc_mean', 'spo2_mean',
                 'baseexcess_mean', 'totalco2_mean', 'calcium_mean', 'lactate_mean', 
       'pco2_mean', 'ph_mean', 'po2_mean',
       'coronary heart disease.csv', 'diabetes.csv', 'family history of stroke.csv', 
       'creatinine.csv_mean',
       'glucose.csv_mean', 'platelet.csv_mean', 'potassium.csv_mean',
       'sodium.csv_mean', 'urea nitrogen.csv_mean', 'WBC.csv_mean', 'aniongap',
       'bicarbonate', 'hematocrit', 'hemoglobin', 'ptt', 'inr', 'pt', 'BMI']
outcome_variables=['liver_dysfunction','thrombocytopenia','hospital_mortality',
                  'septic_shock']
#outcome_variables=['liver_dysfunction']
data=pd.read_csv('result/trainAndTestV6.csv')
split_percentage=0.8
sub_data=data[list(train_variables)+outcome_variables]
train=sub_data[:int(len(data)*split_percentage)]
test=sub_data[int(len(data)*split_percentage):]

FileNotFoundError: [Errno 2] No such file or directory: 'result/trainAndTestV6.csv'

In [ ]:
# XGBoost
report=[]
count=0

min_child_weight_list=[0.2]
for mark1,scale_pos_weight in enumerate([3000]):
    for mark2,n_estimators in enumerate([30]):
        for mark3,max_depth in enumerate([3]):
            for mark4,reg_lambda in enumerate([0.1]):
                for mark5,gamma in enumerate([0.1]):
                    for mark6, min_child_weight in enumerate(min_child_weight_list):
                        for mark7, subsample in enumerate([0.3]):
                            for mark8, max_delta_step in enumerate([3]):
                                for i in outcome_variables:
                                    count=count+1
                                    parameters=[{'n_estimators': [n_estimators], 
                                                'max_depth': [max_depth],
                                                'reg_lambda':[reg_lambda],
                                                'gamma':[gamma],
                                                'scale_pos_weight':[scale_pos_weight]}]
                                    #model=GridSearchCV(XGBClassifier(n_jobs=-1), parameters)
                                    model=XGBClassifier(n_jobs=-1, n_estimators=n_estimators, max_depth=max_depth,
                                                      reg_lambda=reg_lambda,gamma=gamma,scale_pos_weight=scale_pos_weight,
                                                       min_child_weight=min_child_weight,
                                                       subsample=subsample,max_delta_step=max_delta_step)
                                    model.fit(train[train_variables], train[i])
                                    y_score=model.predict_proba(test[train_variables])[:,1]
                                    fpr, tpr, _ = roc_curve(test[i], y_score)
                                    AUROC=auc(fpr, tpr)
                                    trainS=model.score(train[train_variables], train[i])
                                    testS=model.score(test[train_variables], test[i])
                                    f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
                                    preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
                                    recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
                                    AUROC=auc(fpr, tpr)
                                    feature_importance=pd.DataFrame({'column':train_variables, 'values':model.feature_importances_})
                                    feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
                                    report.append([trainS, testS, f1S, preS,recS,AUROC, i,
                                                  feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
                                                  feature_importance.iloc[3].values,feature_importance.iloc[4].values,scale_pos_weight,
                                                  n_estimators,max_depth,reg_lambda,gamma,min_child_weight,
                                                  subsample,max_delta_step])
                                    print(i,':',count)
                                    print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-', 
                                          scale_pos_weight,n_estimators,max_depth,reg_lambda,gamma,min_child_weight)
                                    print('---------------------------------')
                                    joblib.dump(model, 'result/trained models/XGB_models/'+i+'_XGB.model')
    res=pd.DataFrame(report)
    res.to_csv('result/final-test-report.csv')
    